In [1]:
import pandas as pd
import subprocess
import re
from IPython.display import clear_output
import pathlib
from tqdm import tqdm
from clang.cindex import *

Config.set_library_path('/mnt/md0/user/scheuererra68323/clang/llvm-project/build/lib')

CODE_INPUT_DIR = r'/mnt/md0/user/scheuererra68323/testset_jtt/C/testcases'

In [2]:
# find source code files

source_file_paths = []

for ext in ('*.c', '*.cxx', '*.cpp'):
    source_file_paths.extend( pathlib.Path(CODE_INPUT_DIR).rglob(ext) )

print("found: {} source files".format(len(source_file_paths)))

found: 101231 source files


In [3]:
num_found = 0
node_types = []
code_samples = pd.DataFrame(columns=['path', 'line_start', 'line_stop', 'code_snippet']) 

show_progress = True

i = 1
n = len(source_file_paths)   # insert number of relevant files here

tqdm.pandas(desc="Extract Functions")
for path in tqdm(source_file_paths):

    code = open(path, encoding='utf-8').read().split('\n')
    
    index = Index.create()
    tu = index.parse(path)
    for node in tu.cursor.walk_preorder():
        if str(node.location.file) != str(path):
            #print(str(node.location.file) ,"!=", str(path))
            pass
        elif node.kind in [CursorKind.FUNCTION_DECL, 
                           CursorKind.CXX_METHOD, 
                           CursorKind.CONSTRUCTOR,
                           CursorKind.DESTRUCTOR] and node.get_definition() is not None:
            # found a function declaration -> extract the function body
            num_found+=1
            line_start = node.get_definition().extent.start.line
            line_stop  = node.get_definition().extent.end.line
            
            #print ("###{} -- {}-{}".format(node.kind, line_start, line_stop))
            
            # append relevant code lines
            code_snippet_lines = code[line_start-1:line_stop]
            code_snippet = '\n'.join(code_snippet_lines)
            
            #print(code_snippet)

            # find first and last curly braces
            try:
                i1 = code_snippet.index('{')
                i2 = code_snippet.rindex('}')
            except:
                break
                
            code_snippet_lines = code_snippet[i1+1:i2].strip().split('\n')

            #print( code_snippet_lines )

            if code_snippet != '':
                code_samples = code_samples.append({
                    'path' :        pathlib.Path.as_posix(path),   # convert path to posix style before saving
                    'line_start' :  line_start,
                    'line_stop' :   line_stop,
                    'code_snippet': code_snippet_lines
                }, ignore_index=True)
                
            else:
                pass
        
        
print(num_found)
code_samples#.code_snippet

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
  2%|▏         | 1662/101231 [02:14<2:41:34, 10.27it/s]

KeyboardInterrupt: 

## Save to HDF

In [ ]:
code_samples.to_hdf('/mnt/md0/user/scheuererra68323/JTT_ExtractFunctions.h5', key='JTT_ExtractFunctions')

### Determine dataset from filepath

In [ ]:
df = code_samples.copy()

df['dataset'] = df.path.str.extract('code_input\/([^/]+)/')
df.groupby('dataset').count()